In [12]:
import sys
import os
import pandas as pd
import numpy as np
import json

# Add paths for custom modules
sys.path.append(os.path.abspath("../../zero-shot-absa-quad"))
sys.path.append(os.path.abspath("../../zero-shot-absa-quad/plots"))

* nur 5 bis 10 
* gemma umstrukturieren

In [13]:
# from collections import Counter
from performance_helper import compute_f1_scores_quad, compute_scores_single, merge_aspect_lists
from table_tool import round_numbers
import pandas as pd
import numpy as np
# import itertools
# import shutil
# import io, re
import pandas as pd
import json

In [14]:
N_SEEDS = 5
TASKS = ["tasd", "asqp"]
DATASETS = ["rest15", "rest16", "flightabsa", "coursera", "hotels"]
METHODS = ["llm_Meta-Llama-3.1-8B-Instruct-bnb-4bit", "dlo"]
AUG_TECHNIQUES = ["eda", "qaie"]

raw_dataset_to_formatted = {"rest16": "Rest16", "rest15": "Rest15", "flightabsa": "FlightABSA", "coursera": "OATS Coursera", "hotels": "OATS Hotels"}
format_dataset_to_raw = {"Rest16": "rest16", "Rest15": "rest15", "FlightABSA": "flightabsa", "coursera": "OATS Coursera", "OATS Hotels": "hotels"}
raw_method_to_formatted = {"llm_Meta-Llama-3.1-8B-Instruct-bnb-4bit": "FT-LLama-3-8B", "dlo": "DLO \citep{hu2022improving}"}
format_method_to_raw = {"FT-LLama-3-8B": "llm_Meta-Llama-3.1-8B-Instruct-bnb-4bit", "DLO \citep{hu2022improving}": "dlo"}
raw_aug_to_formatted = {"eda": "EDA", "QAIE": "QAIE", "llm_annotator": "LLM-Annotator"}
format_aug_to_raw = {"EDA": "eda", "-": "-", "LLM-Annotator": "llm_annotator"}

In [15]:
def add_element_scores(loaded_json, task):
    labels = loaded_json["all_labels"]
    preds = loaded_json["all_preds"]
    seed_scores = compute_f1_scores_quad(preds, labels)
    seed_scores_ac = compute_scores_single(preds, labels, "single_ac")
    seed_scores_at = compute_scores_single(preds, labels, "single_at")
    seed_scores_pol = compute_scores_single(preds, labels, "single_pol")

    seed_scores["ac"] = seed_scores_ac
    seed_scores["at"] = seed_scores_at
    seed_scores["pol"] = seed_scores_pol
    if task == "asqp":
        seed_scores_ot = compute_scores_single(preds, labels, "single_ot")
        seed_scores["ot"] = seed_scores_ot
    return seed_scores

In [16]:
def calc_mean(scores):
    averages = {}
    for key in scores[0].keys():
        if isinstance(scores[0][key], dict):  # Falls geschachtelte Dicts vorhanden sind
            averages[key] = {subkey: np.mean([s[key][subkey] for s in scores]) for subkey in scores[0][key]}
        else:
            averages[key] = np.mean([s[key] for s in scores])
    return averages

In [17]:
# 1. Load LLM-annotated fine-tuned scores
scores_llm_ann_train = {}

for dataset in DATASETS:
    for task in TASKS:
        for method in METHODS:
            for fs in [0, 10, 50]:
                for n_ann_ex in ["full"]:

                    scores = []
                    for seed in range(N_SEEDS):
                        with open(
                            f"../_out_fine_tunings/01_llm_annotate_train/{method}_{n_ann_ex}_{task}_{fs}_{dataset}_{seed}.json"
                        ) as f:
                            loaded_json = json.load(f)
                            seed_scores = add_element_scores(loaded_json, task)
                            scores.append(seed_scores)
                    scores_llm_ann_train[
                        f"{method}_{n_ann_ex}_{task}_{fs}_{dataset}"
                    ] = calc_mean(scores)

In [18]:
scores_llm_ann_train.keys(
)

dict_keys(['llm_Meta-Llama-3.1-8B-Instruct-bnb-4bit_full_tasd_0_rest15', 'llm_Meta-Llama-3.1-8B-Instruct-bnb-4bit_full_tasd_10_rest15', 'llm_Meta-Llama-3.1-8B-Instruct-bnb-4bit_full_tasd_50_rest15', 'dlo_full_tasd_0_rest15', 'dlo_full_tasd_10_rest15', 'dlo_full_tasd_50_rest15', 'llm_Meta-Llama-3.1-8B-Instruct-bnb-4bit_full_asqp_0_rest15', 'llm_Meta-Llama-3.1-8B-Instruct-bnb-4bit_full_asqp_10_rest15', 'llm_Meta-Llama-3.1-8B-Instruct-bnb-4bit_full_asqp_50_rest15', 'dlo_full_asqp_0_rest15', 'dlo_full_asqp_10_rest15', 'dlo_full_asqp_50_rest15', 'llm_Meta-Llama-3.1-8B-Instruct-bnb-4bit_full_tasd_0_rest16', 'llm_Meta-Llama-3.1-8B-Instruct-bnb-4bit_full_tasd_10_rest16', 'llm_Meta-Llama-3.1-8B-Instruct-bnb-4bit_full_tasd_50_rest16', 'dlo_full_tasd_0_rest16', 'dlo_full_tasd_10_rest16', 'dlo_full_tasd_50_rest16', 'llm_Meta-Llama-3.1-8B-Instruct-bnb-4bit_full_asqp_0_rest16', 'llm_Meta-Llama-3.1-8B-Instruct-bnb-4bit_full_asqp_10_rest16', 'llm_Meta-Llama-3.1-8B-Instruct-bnb-4bit_full_asqp_50_rest16

In [19]:
# 2. Load Augmented fine-tuned scores
scores_traditional_aug = {}

for dataset in DATASETS:
    for task in TASKS:
        for fs in [10, 50]:
                scores = []
                for seed in range(N_SEEDS):
                    path = f"../../QAIE-ABSA-2025-adaption/03_results/{task}_{dataset}_fs_{fs}_{seed}.json"

                    with open(path) as f:
                        loaded_json = json.load(f)
                        seed_scores = add_element_scores(loaded_json, task)
                        scores.append(seed_scores)
                scores_traditional_aug[
                    f"qaie_{task}_{fs}_{dataset}"
                ] = calc_mean(scores)

In [20]:
for dataset in DATASETS:
    for task in TASKS:
        for aug in ["eda"]:
            for method in METHODS:
                for fs in [10, 50]:
                    for n_ann_ex in [2, 5, 10]:
                        scores = []
                        for seed in range(N_SEEDS):
                            path = f"../_out_fine_tunings/03_traditional_augmentation/{method}_{aug}_{n_ann_ex}_{task}_{fs}_{dataset}_{seed}.json"
                            with open(
                                path
                            ) as f:
                                loaded_json = json.load(f)
                                seed_scores = add_element_scores(loaded_json, task)
                                scores.append(seed_scores)
                        scores_traditional_aug[
                            f"{method}_{aug}_{n_ann_ex}_{task}_{fs}_{dataset}"
                        ] = calc_mean(scores)

In [21]:
# 4. Load methods baselines
scores_00_baseline = {}

with open("../../zero-shot-absa-quad/plots/past_results.json") as f:
    past_results = json.load(f)

for dataset in DATASETS:
    for task in TASKS:
        for method in METHODS:
            for n_ann_ex in [10, 50, "full"]:

                scores = []
                for seed in range(N_SEEDS):
                    if n_ann_ex == "full":
                        file_path = f"../../zero-shot-absa-quad/generations/00_baselines/training_{task}_{dataset}_seed-{seed}_n-train_{method}.json"
                    else:
                        file_path = f"../../zero-shot-absa-quad/generations/00_baselines/training_{task}_{dataset}_seed-{seed}_n-train_{method}_{n_ann_ex}.json"
                    with open(file_path) as f:
                        loaded_json = json.load(f)
                        seed_scores = add_element_scores(loaded_json, task)
                        scores.append(seed_scores)
                scores_mean = calc_mean(scores)

                scores_00_baseline[f"{method}_{n_ann_ex}_{task}_{dataset}"] = (
                    scores_mean
                )

for dataset in DATASETS:
    for task in TASKS:
        for method in METHODS:
            for n_ann_ex in ["full"]:
                for metric in ["f1", "precision", "recall"]:
                    try:
                            scores_00_baseline[f"{method}_{n_ann_ex}_{task}_{dataset}"] = (past_results[task][method][dataset])
                            print("successfully loaded past results for",
                                  f"{method}_{n_ann_ex}_{task}_{dataset}_{metric}:", past_results[task][method][dataset][metric])
                    except:
                            pass

successfully loaded past results for dlo_full_tasd_rest15_f1: 62.95
successfully loaded past results for dlo_full_tasd_rest15_precision: -
successfully loaded past results for dlo_full_tasd_rest15_recall: -
successfully loaded past results for dlo_full_asqp_rest15_f1: 48.18
successfully loaded past results for dlo_full_asqp_rest15_precision: 47.08
successfully loaded past results for dlo_full_asqp_rest15_recall: 49.33
successfully loaded past results for dlo_full_tasd_rest16_f1: 71.79
successfully loaded past results for dlo_full_tasd_rest16_precision: -
successfully loaded past results for dlo_full_tasd_rest16_recall: -
successfully loaded past results for dlo_full_asqp_rest16_f1: 59.79
successfully loaded past results for dlo_full_asqp_rest16_precision: 57.92
successfully loaded past results for dlo_full_asqp_rest16_recall: 61.8


In [22]:
# 5. Load zero-shot scores
scores_zeroshot = {}

for dataset in DATASETS:
    for task in TASKS:
        for fs in [0, 10, 20, 30, 40, 50]:
                scores = []
                for seed in range(N_SEEDS):
                    with open(
                        f"../../zero-shot-absa-quad/generations/zeroshot/{task}_{dataset}_test_gemma3:27b_{seed}_label_{fs}.json"
                    ) as f:
                        loaded_json_raw = json.load(f)

                        loaded_json = {
                            "all_preds": [j["pred_label"] for j in loaded_json_raw],
                            "all_labels": [j["tuple_list"] for j in loaded_json_raw],
                        }

                        seed_scores = add_element_scores(loaded_json, task)

                        scores.append(seed_scores)
                scores_zeroshot[f"{task}_{fs}_{dataset}"] = calc_mean(scores)

# WITH SELF-Consistency
for dataset in DATASETS:
    for task in TASKS:
        for fs in [0, 10, 20, 30, 40, 50]:
                all_example_data = []
                for seed in range(N_SEEDS):
                    with open(
                        f"../../zero-shot-absa-quad/generations/zeroshot/{task}_{dataset}_test_gemma3:27b_{seed}_label_{fs}.json"
                    ) as f:
                        loaded_json_raw = json.load(f)

                        loaded_json = {
                            "all_preds": [j["pred_label"] for j in loaded_json_raw],
                            "all_labels": [j["tuple_list"] for j in loaded_json_raw],
                        }

                        all_example_data.append(loaded_json)

                all_labels = all_example_data[0]["all_labels"]
                all_preds = [[] for _ in range(len(all_labels))]
                for seed in range(0, N_SEEDS):
                    for idx in range(len(all_labels)):
                        all_preds[idx].append(all_example_data[seed]["all_preds"][idx])
                        if seed == N_SEEDS - 1:
                            all_preds[idx] = merge_aspect_lists(all_preds[idx])
                            all_preds[idx] = [list(p) for p in all_preds[idx]]

                loaded_json = {
                    "all_preds": all_preds,
                    "all_labels": all_labels,
                }

                scores = add_element_scores(loaded_json, task)
                scores_zeroshot[f"{task}_{fs}_{dataset}_sc"] = scores

In [23]:
def get_n_train_qaie(task="tasd", dataset="rest16", fs=2):
    path = f"../../QAIE-ABSA-2025-adaption/01_augmentations/fs_examples/{task}/{dataset}/fs_{fs}/aug.txt"
    # count number of lines in the file
    with open(path, "r") as f:
        lines = f.readlines()
        n_train = len(lines)
    return n_train

In [24]:
print(scores_zeroshot.keys())
print(scores_00_baseline.keys())
print(scores_llm_ann_train.keys())
print(scores_traditional_aug.keys())

dict_keys(['tasd_0_rest15', 'tasd_10_rest15', 'tasd_20_rest15', 'tasd_30_rest15', 'tasd_40_rest15', 'tasd_50_rest15', 'asqp_0_rest15', 'asqp_10_rest15', 'asqp_20_rest15', 'asqp_30_rest15', 'asqp_40_rest15', 'asqp_50_rest15', 'tasd_0_rest16', 'tasd_10_rest16', 'tasd_20_rest16', 'tasd_30_rest16', 'tasd_40_rest16', 'tasd_50_rest16', 'asqp_0_rest16', 'asqp_10_rest16', 'asqp_20_rest16', 'asqp_30_rest16', 'asqp_40_rest16', 'asqp_50_rest16', 'tasd_0_flightabsa', 'tasd_10_flightabsa', 'tasd_20_flightabsa', 'tasd_30_flightabsa', 'tasd_40_flightabsa', 'tasd_50_flightabsa', 'asqp_0_flightabsa', 'asqp_10_flightabsa', 'asqp_20_flightabsa', 'asqp_30_flightabsa', 'asqp_40_flightabsa', 'asqp_50_flightabsa', 'tasd_0_coursera', 'tasd_10_coursera', 'tasd_20_coursera', 'tasd_30_coursera', 'tasd_40_coursera', 'tasd_50_coursera', 'asqp_0_coursera', 'asqp_10_coursera', 'asqp_20_coursera', 'asqp_30_coursera', 'asqp_40_coursera', 'asqp_50_coursera', 'tasd_0_hotels', 'tasd_10_hotels', 'tasd_20_hotels', 'tasd_30

In [25]:
# Configuration
FT_APPROACHES = ["Llama-3-8B FT", "DLO"]
FT_ENCODING = {"FT-LLama-3-8B": "llm_Meta-Llama-3.1-8B-Instruct-bnb-4bit", "DLO": "dlo", "Llama-3-8B FT": "llama"}
FT_ENCODING_REVERSE = {v: k for k, v in FT_ENCODING.items()}

N_TRAIN_EDA = [5, 10]
N_SHOTS = [10, 50]

def get_mean_n_train(task, fs):
    """Calculate mean n_train across datasets for a given task and fs."""
    return str(np.round(np.mean([
        get_n_train_qaie(task=task, dataset=ds, fs=fs) 
        for ds in DATASETS
    ]), 1))

def create_columns():
    """Create all column data for the DataFrame."""
    # Base pattern for one shot value
    base_pattern_size = len(FT_APPROACHES) * 2 + len(N_TRAIN_EDA) * len(FT_APPROACHES) + 1
    
    # Annotated examples column
    n_annotated = ([0] * 3 + 
                  [10] * (1 + base_pattern_size) + 
                  [50] * (1 + base_pattern_size) + 
                  ["full"] * len(FT_APPROACHES))
    
    # Approaches column - base pattern
    base_approaches = ([["k-shot Gemma"]] + FT_APPROACHES + 
                      [f"LLMA \\textbackslash w {approach}" for approach in FT_APPROACHES] +
                      [f"EDA \\textbackslash w {approach}" for approach in FT_APPROACHES for _ in N_TRAIN_EDA] +
                      ["QAIE"])
    
    approaches = ([["0-shot Gemma"]]+[f"LLMA \\textbackslash w {approach}" for approach in FT_APPROACHES] +
                 base_approaches * 2 +  # For 10 and 50 shots
                 FT_APPROACHES )
    
    # Training columns
    def build_train_column(task):
        column = ["0", "full", "full"] 
        for fs in N_SHOTS:
            column.extend([fs,
                *([fs] * len(FT_APPROACHES)),
                *(["full"] * len(FT_APPROACHES)),
                *([fs + n * fs for _ in FT_APPROACHES for n in N_TRAIN_EDA]),
                get_mean_n_train(task, fs)
            ])
        column.extend(["full"] * len(FT_APPROACHES))
        return column
    
    return n_annotated, approaches, build_train_column("tasd"), build_train_column("asqp")

def collect_performance_scores(tasks, metrics):
    """Collect all performance scores from different score dictionaries."""
    scores = {dataset: {task: {metric: [] for metric in metrics} for task in tasks} 
              for dataset in DATASETS}
    
    for dataset in DATASETS:
        for task in tasks:
            for metric in metrics:
                task_scores = scores[dataset][task][metric]

                task_scores.extend([
                    scores_zeroshot[f"{task}_{fs}_{dataset}_sc"][metric]
                    for fs in [0]
                ])
                
                # Zero-shot scores
                task_scores.extend([
                    scores_llm_ann_train[f"{method}_full_{task}_0_{dataset}"][metric]
                    for method in METHODS
                ])
                
                # Few-shot scores (10, 50)
                for fs in N_SHOTS:
                    # Baseline scores

                    task_scores.extend([
                      scores_zeroshot[f"{task}_{k}_{dataset}_sc"][metric]
                      for k in [fs]
                    ])
                
                    task_scores.extend([
                        scores_00_baseline[f"{method}_{fs}_{task}_{dataset}"][metric]
                        for method in METHODS
                    ])
                    
                    # LLM annotation scores
                    task_scores.extend([
                        scores_llm_ann_train[f"{method}_full_{task}_{fs}_{dataset}"][metric]
                        for method in METHODS
                    ])
                    
                    # Traditional augmentation scores
                    task_scores.extend([
                        scores_traditional_aug[f"{method}_eda_{n_train}_{task}_{fs}_{dataset}"][metric]
                        for method in METHODS
                        for n_train in N_TRAIN_EDA
                    ])
                    
                    # QAIE scores
                    task_scores.append(
                        scores_traditional_aug[f"qaie_{task}_{fs}_{dataset}"][metric]
                    )
                
                # Full training scores
                task_scores.extend([
                    scores_00_baseline[f"{method}_full_{task}_{dataset}"][metric]
                    for method in METHODS
                ])
            
    
    return scores

def create_f1_plot(tasks=["tasd"], metrics=["f1"]):
    """Create F1 plot DataFrame with simplified logic."""
    
    # Get all column data
    n_annotated, approaches, n_train_tasd, n_train_asqp = create_columns()
    
    # Collect performance scores
    performance_scores = collect_performance_scores(tasks, metrics)
    
    # Create DataFrame
    df_data = {
        "\# Annotated examples": n_annotated,
        "Approach": approaches,
        "\# n_train_column_tasd": n_train_tasd,
        "\# n_train_column_asqp": n_train_asqp,
    }
    
    # Add performance columns
    for dataset in DATASETS:
        for task in tasks:
            for metric in metrics:
                df_data[f"{dataset}_{task}_{metric}"] = performance_scores[dataset][task][metric]
    
    df = pd.DataFrame(df_data)
    
    # Round numbers in performance columns
    performance_columns = [f"{dataset}_{task}_{metric}" 
                          for dataset in DATASETS 
                          for task in tasks 
                          for metric in metrics]
    
    df = round_numbers(df, performance_columns, n_rest=2)
    return df

In [26]:
import pandas as pd
import numpy as np

def highlight(df, start_column, end_column, groups, type="underline"):
    """
    Macht die größten Werte in jeder Gruppe für jede Spalte fett.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Der Input DataFrame
    start_column : int
        Index der ersten betroffenen Spalte
    end_column : int
        Index der letzten betroffenen Spalte (inklusive)
    groups : list of lists
        Liste von Gruppen, wobei jede Gruppe eine Liste von Zeilenindizes ist
        z.B. [[0, 1, 5], [2, 3, 4]]
    
    Returns:
    --------
    pd.DataFrame
        DataFrame mit fett markierten Maximalwerten
    """
    # Kopie des DataFrames erstellen
    result_df = df.copy()
    
    # Durch alle betroffenen Spalten iterieren
    for col_idx in range(start_column, end_column + 1):
        col_name = df.columns[col_idx]
        
        # Durch alle Gruppen iterieren
        for group in groups:
            # Werte der aktuellen Gruppe in der aktuellen Spalte
            group_series = df.iloc[group, col_idx]
            
            # Versuche, die Werte in float zu konvertieren, nicht konvertierbare zu NaN
            numeric_values = pd.to_numeric(group_series, errors='coerce')
            
            # Nur gültige (nicht-NaN) Werte berücksichtigen
            if numeric_values.notna().any():
                max_value = numeric_values.max()
                max_indices = numeric_values[numeric_values == max_value].index
                
                # Maximalwerte fett machen
                for idx in max_indices:
                    current_value = result_df.iloc[idx, col_idx]
                    if type == "bold":
                        result_df.iloc[idx, col_idx] = f"\\textbf{{{current_value}}}"
                    elif type == "underline":
                        result_df.iloc[idx, col_idx] = f"\\underline{{{current_value}}}"
    
    return result_df


In [27]:
# load txt file in 01_muster_tex/performance.txt
with open("01_muster_tex/performance.txt", "r") as f:
    template = f.read()

for metric in ["f1", "precision", "recall"]:
    table_out = create_f1_plot(tasks=["tasd", "asqp"], metrics=[metric])
    
    table_out = highlight(table_out, start_column=4, end_column=13, groups=[[0, 1, 2], [k for k in range(3, 13)], [k for k in range(13, 23)]], type="bold")
    table_out = highlight(table_out, start_column=4, end_column=13, groups=[[23, 24]], type="underline")
    
    table_out_list = table_out.iloc[:, 2:].astype(str).values.tolist()
    table_out_list = [item for sublist in table_out_list for item in sublist]

    table_with_values = template
    for i in range(len(table_out_list)):
        table_with_values = table_with_values.replace("xxxx", table_out_list[i], 1)
    

    
    # Write the final table to a file
    with open(f"_out_table/{metric}_table.txt", "w") as f_out:
        f_out.write(table_with_values)

In [28]:
[k for k in range(14, 23)]

[14, 15, 16, 17, 18, 19, 20, 21, 22]

In [29]:
table_out

,\# Annotated examples,Approach,\# n_train_column_tasd,\# n_train_column_asqp,rest15_tasd_recall,rest15_asqp_recall,rest16_tasd_recall,rest16_asqp_recall,flightabsa_tasd_recall,flightabsa_asqp_recall,coursera_tasd_recall,coursera_asqp_recall,hotels_tasd_recall,hotels_asqp_recall
0,0,[0-shot Gemma],0,0,\textbf{31.36},\textbf{26.29},46.57,30.29,56.90,45.42,32.58,15.14,41.02,23.16
1,0,LLMA \textbackslash w Llama-3-8B FT,full,full,27.72,21.46,48.61,\textbf{30.86},\textbf{57.96},\textbf{48.54},33.20,15.50,42.89,25.60
2,0,LLMA \textbackslash w DLO,full,full,27.36,20.91,\textbf{49.27},29.81,56.98,47.39,\textbf{34.47},\textbf{16.02},\textbf{43.85},\textbf{26.21}
3,10,[k-shot Gemma],10,10,\textbf{52.66},\textbf{40.50},\textbf{65.19},47.93,60.87,45.08,\textbf{40.37},21.31,\textbf{55.17},28.29
4,10,Llama-3-8B FT,10,10,30.56,15.72,38.74,18.90,34.71,20.10,21.15,8.76,32.46,16.06
5,10,DLO,10,10,13.47,4.13,13.95,4.91,13.91,4.03,20.86,4.02,17.39,3.41
6,10,LLMA \textbackslash w Llama-3-8B FT,full,full,52.12,39.42,61.70,45.78,61.32,46.51,38.52,23.94,53.16,\textbf{30.04}
7,10,LLMA \textbackslash w DLO,full,full,48.19,38.99,61.28,\textbf{48.81},\textbf{61.51},\textbf{47.36},39.63,\textbf{23.98},54.50,29.93
8,10,EDA \textbackslash w Llama-3-8B FT,60,60,38.49,16.83,41.26,22.38,28.88,21.97,28.03,13.82,35.64,18.95
9,10,EDA \textbackslash w Llama-3-8B FT,110,110,37.04,13.48,38.84,20.83,27.26,20.17,29.59,12.51,32.88,17.89
